In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
df = pd.read_pickle('df_all_movies_2018_pk')
df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Black Panther,"Feb 16, 2018",Walt Disney,Action,"$700,059,566","76,845,177",https://www.the-numbers.com/movie/Black-Panthe...,https://www.the-numbers.com/movie/Black-Panthe...
1,2,Avengers: Infinity War,"Apr 27, 2018",Walt Disney,Action,"$678,815,482","74,513,225",https://www.the-numbers.com/movie/Avengers-Inf...,https://www.the-numbers.com/movie/Avengers-Inf...
2,3,Incredibles 2,"Jun 15, 2018",Walt Disney,Adventure,"$608,581,744","66,803,704",https://www.the-numbers.com/movie/Incredibles-...,https://www.the-numbers.com/movie/Incredibles-...
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018",Universal,Action,"$417,719,760","45,852,883",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...
4,5,Deadpool 2,"May 18, 2018",20th Century Fox,Action,"$324,512,774","35,621,600",https://www.the-numbers.com/movie/Deadpool-2#t...,https://www.the-numbers.com/movie/Deadpool-2#t...
...,...,...,...,...,...,...,...,...,...
731,732,To Die is Hard,"Dec 17, 2010",Driving With Our …,Comedy,$580,63,https://www.the-numbers.com/movie/To-Die-is-Ha...,https://www.the-numbers.com/movie/To-Die-is-Ha...
732,733,Higher Power,"May 11, 2018",Magnet Pictures,Thriller/Suspense,$528,57,https://www.the-numbers.com/movie/Higher-Power...,https://www.the-numbers.com/movie/Higher-Power...
733,734,Mobile Homes,"Apr 20, 2018",Mongrel Media,Drama,$449,49,https://www.the-numbers.com/movie/Mobile-Homes...,https://www.the-numbers.com/movie/Mobile-Homes...
734,735,Midget Zombie Takeover,"Feb 8, 2013",Driving With Our …,Comedy,$402,44,https://www.the-numbers.com/movie/Midget-Zombi...,https://www.the-numbers.com/movie/Midget-Zombi...


In [3]:
row_count = df.shape[0]
row_count

736

In [5]:
i = 0
r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
s = bs(r.text)

ex_all_tables = s.find_all('table')
ex_row = ex_all_tables[2].find_all('tr')
ex_row

[<tr><td><b>Opening Weekend:</b></td><td>$202,003,951 (28.9% of total gross)</td></tr>,
 <tr><td><b>Legs:</b></td><td>3.47 (domestic box office/biggest weekend)</td></tr>,
 <tr><td><b>Domestic Share:</b></td><td>52.4% (domestic box office/worldwide)</td></tr>,
 <tr><td><b>Production Budget:</b></td><td>$200,000,000 (worldwide box office is 6.7 times production budget)</td></tr>,
 <tr><td><b>Theater counts:</b></td><td>4,020 opening theaters/4,084 max. theaters, 9.9 weeks average run per theater</td></tr>,
 <tr>
 <td><b>Infl. Adj. Dom. BO</b></td>
 <td>$703,901,821</td>
 </tr>]

In [6]:
# Extract legs trial
ex_table_metrics = ex_all_tables[2]
all_rows_metrics = ex_table_metrics.find_all('tr')
for row in all_rows_metrics:
    found_legs = False
    if ('Legs' in row.text):
        ex_rows_legs = row
        found_legs = True
        break

if (found_legs):
    ex_strg_legs = ex_rows_legs.text
    print(ex_strg_legs)

    pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
    print(pattern.findall(ex_strg_legs))
    legs = float(pattern.findall(ex_strg_legs)[0])
else:
    legs = 'not_found'
        
legs

Legs:3.47 (domestic box office/biggest weekend)
['3.47']


3.47

In [7]:
# Test Extract production budget 

ex_row_budget = ex_table_metrics.find_all('tr')[3]
ex_strg_budget = ex_row_budget.text
print(ex_strg_budget)

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

print(pattern.findall(ex_strg_budget))
budget_ratio_list = pattern.findall(ex_strg_budget)


ratio = float(budget_ratio_list.pop())
print(ratio)

budget_str = ''
budget_str = budget_str.join(budget_ratio_list)
print(budget_str)

budget = int(budget_str)
budget

Production Budget:$200,000,000 (worldwide box office is 6.7 times production budget)
['200', '000', '000', '6.7']
6.7
200000000


200000000

In [8]:
#extract theater count trial

ex_row_theater_count = ex_table_metrics.find_all('tr')[4]
ex_strg_theater_count = ex_row_theater_count.find_all('td')[1].text

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
print(pattern.findall(ex_strg_theater_count))
theater_count_list = pattern.findall(ex_strg_theater_count)

theater_count = int(theater_count_list[0] + theater_count_list[1])
print(theater_count)

theater_run = float(theater_count_list[-1])
print(theater_run)

['4', '020', '4', '084', '9.9']
4020
9.9


In [9]:
#Extract Movie Details table
ex_table_movie_details = ex_all_tables[5]

In [10]:
#Extract franchise trial

ex_row_franchise = ex_table_movie_details.find_all('tr')[5]
ex_elems = ex_row_franchise.find_all('td')[1].find_all('a')
for i in ex_elems:
    print(i.text)

Marvel Cinematic Universe


In [11]:
#Extract source trial
ex_row_source = ex_table_movie_details.find_all('tr')[8]
source = ex_row_source.find_all('td')[1].find_all('a')[0].text
source

'Based on Comic/Graphic Novel'

In [12]:
#Extract Genre trial
ex_row_genre = ex_table_movie_details.find_all('tr')[9]
genre = ex_row_genre.find_all('td')[1].find_all('a')[0].text
genre

'Action'

In [13]:
# Extract creative type trial
ex_row_ctype = ex_table_movie_details.find_all('tr')[11]
ctype = ex_row_ctype.find_all('td')[1].find_all('a')[0].text
ctype

'Super Hero'

In [14]:
# Extract legs for all 2018 movies  v2

all_movies_legs_list = []
all_movies_all_tables_2018_list = []

for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_legs = False
    
        for row in all_rows:
            if ('Legs' in row.text):
                ex_rows_legs = row
                found_legs = True

                ex_strg_legs = ex_rows_legs.text
                print(ex_strg_legs)

                pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
                print(pattern.findall(ex_strg_legs))
                legs = float(pattern.findall(ex_strg_legs)[0])
                all_movies_legs_list.append(legs)
                break        
        if (found_legs):
            break
                
    if (not found_legs):
        legs = 1
        all_movies_legs_list.append(legs)
    
    print(legs)
    

legs_2018_df = pd.DataFrame(all_movies_legs_list, columns = ['Legs'])
legs_2018_df.to_pickle('legs_2018_df_pk')



Legs:3.47 (domestic box office/biggest weekend)
['3.47']
3.47
Legs:2.63 (domestic box office/biggest weekend)
['2.63']
2.63
Legs:3.33 (domestic box office/biggest weekend)
['3.33']
3.33
Legs:2.82 (domestic box office/biggest weekend)
['2.82']
2.82
Legs:2.59 (domestic box office/biggest weekend)
['2.59']
2.59
Legs:4.01 (domestic box office/biggest weekend)
['4.01']
4.01
Legs:8.08 (domestic box office/biggest weekend)
['8.08']
8.08
Legs:3.60 (domestic box office/biggest weekend)
['3.60']
3.6
Legs:2.86 (domestic box office/biggest weekend)
['2.86']
2.86
Legs:2.53 (domestic box office/biggest weekend)
['2.53']
2.53
Legs:2.66 (domestic box office/biggest weekend)
['2.66']
2.66
Legs:5.02 (domestic box office/biggest weekend)
['5.02']
5.02
Legs:4.94 (domestic box office/biggest weekend)
['4.94']
4.94
Legs:4.24 (domestic box office/biggest weekend)
['4.24']
4.24
Legs:3.75 (domestic box office/biggest weekend)
['3.75']
3.75
Legs:3.58 (domestic box office/biggest weekend)
['3.58']
3.58
Legs:6.58

Legs:12.56 (domestic box office/biggest weekend)
['12.56']
12.56
Legs:2.17 (domestic box office/biggest weekend)
['2.17']
2.17
Legs:4.50 (domestic box office/biggest weekend)
['4.50']
4.5
Legs:8.36 (domestic box office/biggest weekend)
['8.36']
8.36
Legs:2.56 (domestic box office/biggest weekend)
['2.56']
2.56
Legs:2.64 (domestic box office/biggest weekend)
['2.64']
2.64
Legs:13.98 (domestic box office/biggest weekend)
['13.98']
13.98
Legs:5.28 (domestic box office/biggest weekend)
['5.28']
5.28
Legs:2.16 (domestic box office/biggest weekend)
['2.16']
2.16
Legs:4.81 (domestic box office/biggest weekend)
['4.81']
4.81
Legs:2.59 (domestic box office/biggest weekend)
['2.59']
2.59
Legs:6.16 (domestic box office/biggest weekend)
['6.16']
6.16
Legs:2.39 (domestic box office/biggest weekend)
['2.39']
2.39
Legs:2.85 (domestic box office/biggest weekend)
['2.85']
2.85
Legs:2.33 (domestic box office/biggest weekend)
['2.33']
2.33
Legs:2.43 (domestic box office/biggest weekend)
['2.43']
2.43
Leg

Legs:25.38 (domestic box office/biggest weekend)
['25.38']
25.38
Legs:5.88 (domestic box office/biggest weekend)
['5.88']
5.88
Legs:3.65 (domestic box office/biggest weekend)
['3.65']
3.65
Legs:4.93 (domestic box office/biggest weekend)
['4.93']
4.93
Legs:8.08 (domestic box office/biggest weekend)
['8.08']
8.08
Legs:2.98 (domestic box office/biggest weekend)
['2.98']
2.98
Legs:157.34 (domestic box office/biggest weekend)
['157.34']
157.34
Legs:6.14 (domestic box office/biggest weekend)
['6.14']
6.14
Legs:4.07 (domestic box office/biggest weekend)
['4.07']
4.07
Legs:1.78 (domestic box office/biggest weekend)
['1.78']
1.78
Legs:2.07 (domestic box office/biggest weekend)
['2.07']
2.07
Legs:7.75 (domestic box office/biggest weekend)
['7.75']
7.75
Legs:2.97 (domestic box office/biggest weekend)
['2.97']
2.97
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:2.55 (domestic box office/biggest weekend)
['2.55']
2.55
Legs:16.41 (domestic box office/biggest weekend)
['16.41']
16.

Legs:4.33 (domestic box office/biggest weekend)
['4.33']
4.33
Legs:5.36 (domestic box office/biggest weekend)
['5.36']
5.36
Legs:1.14 (domestic box office/biggest weekend)
['1.14']
1.14
Legs:6.86 (domestic box office/biggest weekend)
['6.86']
6.86
Legs:3.17 (domestic box office/biggest weekend)
['3.17']
3.17
Legs:2.37 (domestic box office/biggest weekend)
['2.37']
2.37
Legs:8.70 (domestic box office/biggest weekend)
['8.70']
8.7
Legs:12.53 (domestic box office/biggest weekend)
['12.53']
12.53
Legs:3.79 (domestic box office/biggest weekend)
['3.79']
3.79
Legs:3.88 (domestic box office/biggest weekend)
['3.88']
3.88
Legs:2.52 (domestic box office/biggest weekend)
['2.52']
2.52
Legs:4.82 (domestic box office/biggest weekend)
['4.82']
4.82
Legs:2.46 (domestic box office/biggest weekend)
['2.46']
2.46
Legs:4.61 (domestic box office/biggest weekend)
['4.61']
4.61
Legs:9.61 (domestic box office/biggest weekend)
['9.61']
9.61
Legs:2.05 (domestic box office/biggest weekend)
['2.05']
2.05
Legs:5

Legs:5.12 (domestic box office/biggest weekend)
['5.12']
5.12
Legs:10.01 (domestic box office/biggest weekend)
['10.01']
10.01
Legs:1.90 (domestic box office/biggest weekend)
['1.90']
1.9
Legs:3.07 (domestic box office/biggest weekend)
['3.07']
3.07
Legs:11.16 (domestic box office/biggest weekend)
['11.16']
11.16
Legs:3.81 (domestic box office/biggest weekend)
['3.81']
3.81
Legs:9.00 (domestic box office/biggest weekend)
['9.00']
9.0
Legs:9.49 (domestic box office/biggest weekend)
['9.49']
9.49
Legs:2.84 (domestic box office/biggest weekend)
['2.84']
2.84
Legs:5.91 (domestic box office/biggest weekend)
['5.91']
5.91
Legs:5.10 (domestic box office/biggest weekend)
['5.10']
5.1
Legs:3.60 (domestic box office/biggest weekend)
['3.60']
3.6
Legs:2.48 (domestic box office/biggest weekend)
['2.48']
2.48
Legs:2.79 (domestic box office/biggest weekend)
['2.79']
2.79
Legs:18.99 (domestic box office/biggest weekend)
['18.99']
18.99
Legs:2.88 (domestic box office/biggest weekend)
['2.88']
2.88
Leg

Legs:5.22 (domestic box office/biggest weekend)
['5.22']
5.22
Legs:8.01 (domestic box office/biggest weekend)
['8.01']
8.01
Legs:1.94 (domestic box office/biggest weekend)
['1.94']
1.94
Legs:1.79 (domestic box office/biggest weekend)
['1.79']
1.79
Legs:1.98 (domestic box office/biggest weekend)
['1.98']
1.98
Legs:2.19 (domestic box office/biggest weekend)
['2.19']
2.19
Legs:6.62 (domestic box office/biggest weekend)
['6.62']
6.62
Legs:1.69 (domestic box office/biggest weekend)
['1.69']
1.69
Legs:2.64 (domestic box office/biggest weekend)
['2.64']
2.64
Legs:1.65 (domestic box office/biggest weekend)
['1.65']
1.65
Legs:2.84 (domestic box office/biggest weekend)
['2.84']
2.84
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:1.99 (domestic box office/biggest weekend)
['1.99']
1.99
Legs:1.48 (domestic box office/biggest weekend)
['1.48']
1.48
Legs:1.77 (domestic box office/biggest weekend)
['1.77']
1.77
Legs:1.77 (domestic box office/biggest weekend)
['1.77']
1.77
Legs:1.50

In [19]:
legs_2018_df

In [31]:
# Extract budget and ratio for all  movies v2

all_movies_budget_list = []
all_movies_wbo_to_budget_ratio_list = []


for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_budget = False
    
        for row in all_rows:
            if ('Budget' in row.text):
    #             print(row.text)
                found_budget = True
                ex_row_budget = row
                ex_strg_budget = ex_row_budget.text
    #             print(ex_strg_budget)

                pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

    #             print(pattern.findall(ex_strg_budget))
                budget_ratio_list = pattern.findall(ex_strg_budget)


                ratio = float(budget_ratio_list.pop())
                print(ratio)

                budget_str = ''
                budget_str = budget_str.join(budget_ratio_list)
                print(budget_str)

                budget = int(budget_str)

                all_movies_budget_list.append(budget)
                all_movies_wbo_to_budget_ratio_list.append(ratio)
                break
        if(found_budget):
            break

    if (not found_budget):
        budget = 'nf'
        ratio = 'nf'
        all_movies_budget_list.append(budget)
        all_movies_wbo_to_budget_ratio_list.append(ratio)
    
    
    print(all_movies_budget_list)
    print(all_movies_wbo_to_budget_ratio_list)
    
    
budget_2018_df = pd.DataFrame(all_movies_budget_list, 
                                  columns = ['Production_Budget'])
wbo_to_budget_ratio_2018_df = pd.DataFrame(all_movies_wbo_to_budget_ratio_list,
                                               columns = ['WBO to Budget Ratio'])
     
budget_2018_df.to_pickle('budget_2018_pk')
wbo_to_budget_ratio_2018_df.to_pickle('wbo_to_budget_ratio_2018_pk')

In [23]:
 budget_2018_df[budget_2018_df['Production_Budget'] == 'nf']

,Production_Budget
76,nf
89,nf
109,nf
123,nf
130,nf
...,...
730,nf
731,nf
733,nf
734,nf


In [32]:
# Extract theater count and average run for all 2018 movies

all_movies_theater_count_list = []
all_movies_avg_run_list = []

for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    count = i

    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_theater = False
        
        for row in all_rows:
            if ('Theater' in row.text and 'counts' in row.text):
                found_theater = True

                ex_row_theater_count = row
                print(row.text)
                ex_strg_theater_count = row.text

                pattern1 = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
                pattern2 = re.compile(r"[-+]?(?:\d*\,\d+|\d+)")
                print(pattern1.findall(ex_strg_theater_count))
                print(pattern2.findall(ex_strg_theater_count))
                
                theater_count_list1 = pattern1.findall(ex_strg_theater_count)
                theater_count_list2 = pattern2.findall(ex_strg_theater_count)
 
                theater_run = float(theater_count_list1[-1])
                print(theater_run)
                theater_count = int(theater_count_list2[0].replace(',', ''))
                print(theater_run)

                all_movies_theater_count_list.append(theater_count)
                all_movies_avg_run_list.append(theater_run)
                break
           
        if(found_theater):
            break     
                
    if(not found_theater):
        theater_count = 'nf'
        theater_run = 'nf'

        all_movies_theater_count_list.append(theater_count)
        all_movies_avg_run_list.append(theater_run)
            
            
theater_count_2018_df = pd.DataFrame(all_movies_theater_count_list,
                                     columns = ['Theater_Count'])
avg_run_2018_df = pd.DataFrame(all_movies_avg_run_list,
                               columns = ['Average_Run'])
       

theater_count_2018_df.to_pickle('theater_count_2018_pk')
avg_run_2018_df.to_pickle('avg_run_2018_pk')

In [29]:
len(all_movies_theater_count_list)
avg_run_2018_df

In [30]:
theater_count_2018_df

,Theater_Count
0,4020
1,4474
2,4410
3,4475
4,4349
...,...
731,1
732,3
733,1
734,1


In [34]:
# Extract source for all 2018 movies

all_movies_source_list = []

for i in range(row_count):
    summary_url = df.loc[i,'summary_page_url']
    print(summary_url)
    
    r = requests.get(summary_url) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_source = False
        for row in all_rows:
    #         print(row)
            if ('Source' in row.text):
                found_source = True
                source_strg = row.find_all('a')[0].text
                print(source_strg)
                all_movies_source_list.append(source_strg)
                break
        
        if (found_source):
            break
    if (not found_source):
        source_strg = 'nf'
        print()
        all_movies_source_list.append(source_strg)
    

source_2018_df = pd.DataFrame(all_movies_source_list, columns = ['Source'])        
source_2018_df.to_pickle('source_2018_pk') 
    

https://www.the-numbers.com/movie/Black-Panther#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Avengers-Infinity-War#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Incredibles-2#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Jurassic-World-Fallen-Kingdom-(2018)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Deadpool-2#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Dr-Seuss-The-Grinch-(2018)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Jumanji-Welcome-to-the-Jungle#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Mission-Impossible-Fallout#tab=summary
Based on TV
https://www.the-numbers.com/movie/Ant-Man-and-the-Wasp#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Solo-A-Star-Wars-Story#tab=summary
Spin-Off
https://www.the-numbers.com/movie/Venom-(2018)#tab=summary
Spin-Off
h

Based on Real Life Events
https://www.the-numbers.com/movie/Ferdinand#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Robin-Hood-(2018)#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Super-Troopers-2#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Slender-Man#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Three-Billboards-Outside-Ebbing-Missouri#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Coco-(2017)#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Teen-Titans-Go-To-The-Movies#tab=summary
Based on TV
https://www.the-numbers.com/movie/Hostiles-(2017)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Vice-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Hate-U-Give-The#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Holmes-an

Original Screenplay
https://www.the-numbers.com/movie/Gotti-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Ya-Veremos-(Mexico)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Simmba-(India)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Life-Itself-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Thor-Ragnarok#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Gosnell-The-Trial-of-America-s-Biggest-Serial-Killer-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Indivisible-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/2018-Oscar-Shorts#tab=summary
Compilation
https://www.the-numbers.com/movie/Disobedience-(2018)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Justice-League-(2017)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/First-Reformed-(

Original Screenplay
https://www.the-numbers.com/movie/Walk-with-me-(UK)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Catcher-Was-A-Spy-The-(2018)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Mirai-no-Mirai-(Japan)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Little-Stranger-The#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Zhuo-yao-ji-2-(China)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Blaze-(2018)#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/Boundaries-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Nada-a-Perder-(Brazil)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Yi-Chu-Hao-Xi-(China)-(2018)#tab=summary

https://www.the-numbers.com/movie/Bombshell-The-Hedy-Lamarr-Story#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Pafekuto

Based on Real Life Events
https://www.the-numbers.com/movie/Happy-End-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Mutafukaz-(France)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Island-of-Lemurs-Madagascar#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Jane-(2017)#tab=summary
Compilation
https://www.the-numbers.com/movie/Harry-Potter-and-the-Prisoner-of-Azkaban#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Bad-Moms-Christmas-A#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Dark-Money-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Getting-Grace#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Dong-Du-Te-Gong-(China)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Skyldige-Den-(Denmark)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Keep-The-Change-(2017)#tab=

Based on Real Life Events
https://www.the-numbers.com/movie/Running-for-Grace-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Sandome-no-satsujin-(Japan)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Price-of-Everything-The-(Documentary)-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Nostalgia-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Da-Shi-Xiong-(Hong-Kong)#tab=summary

https://www.the-numbers.com/movie/Sweet-Country-(Australia)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Long-Xia-Xing-Jing-(China)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/How-to-Talk-to-Girls-at-Parties#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Jusqu-a-la-garde-(France)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/camera-de-Claire-La-(France-2017)#tab=summary
Original Screenplay
https://www.th

Compilation
https://www.the-numbers.com/movie/Holy-Hell-(Canada)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bedoune-Tarikh-Bedoune-Emza-(Iran)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Ingmar-Bergman-Vermachtnis-eines-Jahrhundertgenies-(Germany)-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Gods-Own-Country-(UK)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Just-Getting-Started#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Sollers-Point#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/120-battements-par-minute-(France)-(BPM-Beats-Per-Minute)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Moynihan-(Documentary)-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Saving-Brinton#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/As-Boas-Maneiras-(Brazil)#tab=summary
Original

Based on Real Life Events
https://www.the-numbers.com/movie/My-Art#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Head-Full-of-Honey-(2018)#tab=summary
Based on Movie
https://www.the-numbers.com/movie/Tyrel-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Most-Unknown-The-(Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Retour-du-Heros-Le-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Ex-Libris-The-New-York-Public-Library#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Minding-the-Gap-(Documentary)-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Last-Race-The-(Documentary)-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Victoria-and-Abdul#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/Summer-03-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/

Based on Real Life Events
https://www.the-numbers.com/movie/Nelly-(Canada)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/State-Like-Sleep-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/L-A-Minute-An-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Nana-(2017)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Paris-Opera-The-(Switzerland)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Straight-Into-a-Storm-(Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/12-jours-(France-2017)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/ACORN-and-the-Firestorm-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Shadowman-(2017-Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Tribes-of-Palos-Verdes-The#tab=summary
Based on Fiction Book/Short Story
https://ww

In [38]:
len(all_movies_source_list)
source_2018_df[source_2018_df['Source'] == 'nf']

In [39]:
all_movies_creative_type_list = []

for i in range(row_count):
    summary_url = df.loc[i,'summary_page_url']
    
    r = requests.get(summary_url) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_ctype = False
        for row in all_rows:
#             print(row.text)
            if ('Creative' in row.text and 'Type' in row.text):
                found_ctype = True
#                 print(row)
                ctype = row.find_all('a')[0].text
                print(ctype)
                
                all_movies_creative_type_list.append(ctype)
                break
        
        if (found_ctype):
            break
    if (not found_ctype):
        ctype = 'nf'
        print('df')
        all_movies_creative_type_list.append(ctype)
    

creative_type_2018_df = pd.DataFrame(all_movies_creative_type_list,
                                     columns = ['Creative_Type'])        
creative_type_2018_df.to_pickle('creative_type_2018_pk') 

Super Hero
Super Hero
Kids Fiction
Science Fiction
Super Hero
Kids Fiction
Science Fiction
Contemporary Fiction
Super Hero
Science Fiction
Super Hero
Contemporary Fiction
Super Hero
Dramatization
Science Fiction
Kids Fiction
Contemporary Fiction
Kids Fiction
Contemporary Fiction
Fantasy
Science Fiction
Contemporary Fiction
Kids Fiction
Science Fiction
Super Hero
Dramatization
Contemporary Fiction
Contemporary Fiction
Kids Fiction
Contemporary Fiction
Science Fiction
Contemporary Fiction
Science Fiction
Fantasy
Contemporary Fiction
Kids Fiction
Dramatization
Kids Fiction
Dramatization
Contemporary Fiction
Science Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Dramatization
Contemporary Fiction
Science Fiction
Contemporary Fiction
Science Fiction
Fantasy
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Science Fiction
Contemporary Fiction
Contemporary Fiction
Contempora

Contemporary Fiction
Contemporary Fiction
Factual
Science Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Dramatization
Kids Fiction
Historical Fiction
Contemporary Fiction
Factual
Factual
Factual
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Factual
Fantasy
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Dramatization
Factual
Factual
Contemporary Fiction
Factual
Historical Fiction
Science Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Factual
Factual
Contemporary Fiction
Dramatization
Contemporary Fiction
Contemporary Fiction
Factual
Fantasy
Contemporary Fiction
Factual
Contemporary Fiction
Factual
Fantasy
Contemporary F

In [40]:
len(all_movies_creative_type_list)
creative_type_2018_df

,Creative_Type
0,Super Hero
1,Super Hero
2,Kids Fiction
3,Science Fiction
4,Super Hero
...,...
731,Contemporary Fiction
732,Science Fiction
733,Contemporary Fiction
734,Science Fiction


In [43]:
creative_type_2018_df[creative_type_2018_df['Creative_Type'] == 'nf']

,Creative_Type
231,nf
262,nf
280,nf
292,nf
296,nf
300,nf
385,nf
429,nf
577,nf
590,nf
